# Interacting with the ESCALATE REST API

This is a tutorail for using the ESCALATE REST API. 

It presumes only knowledge of basic python. 

It also presumes you have a local instance of ESCALATE running at http://localhost:8000

In [1]:
import json
from pprint import pprint

import requests  # requests library will send and receive data from the escalate server
from requests.api import post

import pandas as pd

## Quick intro to REST APIs

All you need to know about REST is that:
* It is a protocol for exchanging data between a client (e.g. me) and a server (e.g. ESCALATE)
* Data formats are human *and* machine readable (XML or JSON)
* There are two main HTTP 'verbs' or functions: 
  1. GET data from the server
  2. POST data to the server  
     (there are others, e.g. PUT and PATCH, but we won't use these in this tutorial)
* The python `requests` library implements the HTTP verbs for interacting with servers, including with REST APIs

### Example: GET from the PubChem API

* [Pubchem](https://pubchem.ncbi.nlm.nih.gov/) is a great way to get chemical informatics data from a vast repository
   - e.g. just search for any compound above
   - Has a graphical web interface and REST API
* (There are other computational chemistry REST APIs, including [Open Chemistry](https://doi.org/10.1186/s13321-017-0241-z), and [AFLOW-ML](https://doi.org/10.1016/j.commatsci.2018.03.075))


In [2]:
# dict mapping compound name to PubChemID
PubChemIDs = {
    'Methane': '297',
    'Benzene': '241',
    'Ethylammonium Iodide': '11116533'
}

The URL below is the compound 'API endpoint' which I can send a request to for properties of compounds given their PubChem CIDs

In [3]:
response = requests.get(('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound'          # What type of entity? Compound
                         f'/cid/{",".join(PubChemIDs.values())}/'                      # Which compounds? These
                         'property/MolecularFormula,MolecularWeight,CanonicalSMILES/'  # Which properties? 
                         'JSON'))                                                     # Which format?
                        #proxies=proxies)

In [4]:
response

<Response [200]>

In [5]:
response.json()

{'PropertyTable': {'Properties': [{'CID': 297,
    'MolecularFormula': 'CH4',
    'MolecularWeight': '16.043',
    'CanonicalSMILES': 'C'},
   {'CID': 241,
    'MolecularFormula': 'C6H6',
    'MolecularWeight': '78.11',
    'CanonicalSMILES': 'C1=CC=CC=C1'},
   {'CID': 11116533,
    'MolecularFormula': 'C2H8IN',
    'MolecularWeight': '173.00',
    'CanonicalSMILES': 'CCN.I'}]}}

## The ESCALATE REST API 

* Send data back and forth between ML, chemists, and laboratories

### Interactive List of all available ESCALATE endpoints

Simply navigating to http://localhost:8000/api will show you a browsable list of all available API endpoints. 

We can also view that list programmatically: 

In [6]:
base_url = 'http://localhost:8000/api'  # local dev server
response = requests.get(base_url)
response.json()

{'action': 'http://localhost:8000/api/action/',
 'actiondef': 'http://localhost:8000/api/actiondef/',
 'actionparameter': 'http://localhost:8000/api/actionparameter/',
 'actionparameterdefassign': 'http://localhost:8000/api/actionparameterdefassign/',
 'actor': 'http://localhost:8000/api/actor/',
 'billofmaterials': 'http://localhost:8000/api/billofmaterials/',
 'bomcompositematerial': 'http://localhost:8000/api/bomcompositematerial/',
 'bommaterial': 'http://localhost:8000/api/bommaterial/',
 'calculation': 'http://localhost:8000/api/calculation/',
 'calculationdef': 'http://localhost:8000/api/calculationdef/',
 'compositematerial': 'http://localhost:8000/api/compositematerial/',
 'compositematerialproperty': 'http://localhost:8000/api/compositematerialproperty/',
 'condition': 'http://localhost:8000/api/condition/',
 'conditioncalculationdefassign': 'http://localhost:8000/api/conditioncalculationdefassign/',
 'conditiondef': 'http://localhost:8000/api/conditiondef/',
 'experiment': '

### Logging into ESCALATE


To be able to view and post to all endpoints, I will create a log in session that is managed by a token. 

In [7]:
# demo login credentials
login_data = {
    'username': 'mtynes',
    'password': 'hello1world2'
}

r_login = requests.post(f'{base_url}/login', data=login_data)
token = r_login.json()['token']
token

'f3a03cb5ffa0efa1360e5093f303dd338b4e92a6'

In [8]:
token_header = {'Authorization': f'Token {token}'}

content_type_header = {'content-type': 'application/json'} # for most requests we'll want this header

This token will allow me to validate my identity for this session

### Simple helper functions for GET/POST

These functions will do some minimal URL generation and JSON response parsing on top of what is done by the `requests` library

In [9]:
def post_data(endpoint, data={}, headers={**token_header, **content_type_header}):
    """POST `data` to `endpoint`in ESCALATE API using `headers`
    
    return: (dict|requests.Response), bool
    """
    r = requests.post(f'{base_url}/{endpoint}/', 
                      data=json.dumps(data), 
                      headers=headers)
    print(r)
    if r.ok: 
        print('POST: OK, returning new resource dict')
        return r.json()
    print('POST: FAILED, returning response object')
    return r


def get_data(endpoint, data={}, headers={**token_header}):
    """Make GET request with `data` to `endpoint` in ESCALATE API using `headers`
    
    return: (dict|list|requests.Response), bool
    """
    r = requests.get(f'{base_url}/{endpoint}/', params=data, headers=headers)
    print(r)
    if r.ok: 
        print('GET: OK')
        
        resp_json = r.json()        
        
        # handle cases: one vs many results
        if resp_json.get('count') is None: # edge case: template edit
            return r.json()
        elif resp_json.get('count') == 1: 
            print('Found one resource, returning dict')
            return resp_json['results'][0]
        elif resp_json.get('count') >= 1: 
            print(f"Found {resp_json['count']} resources, returning list of dicts)")
            return r.json()['results']
        else:
            print('GET: FAILED, returning response object')
    return r

### GET all of materials defined in ESCALATE

In [10]:
r = get_data(endpoint='material')

<Response [200]>
GET: OK
Found 294 resources, returning list of dicts)


All of these materials would probably take up too much space in this notebook, lets look at the first one.

All lead iodide shares in this chemical formula, etc. Models are jsut 'what is is about these thigns that are all the same, as opposed to particular vials of gblthat has volumes, contanimanants, dates, masses. these all come from the object, but share in model. models dont contain provenance, are only abotu intenseive properties 

In [11]:
r[0]

{'url': 'http://localhost:8000/api/material/de4b4542-722e-4382-83da-f0b71b9d7f1e/',
 'uuid': 'de4b4542-722e-4382-83da-f0b71b9d7f1e',
 'edocs': [],
 'tags': [],
 'notes': [],
 'description': 'Gamma-Butyrolactone',
 'consumable': True,
 'composite_flg': False,
 'material_class': 'model',
 'actor_description': None,
 'status_description': 'active',
 'add_date': '2021-05-18T18:36:39.541032Z',
 'mod_date': '2021-05-18T18:36:48.001730Z',
 'actor': None,
 'status': 'http://localhost:8000/api/status/5f7417a3-cf9a-425d-b4c5-f72db9c5d702/',
 'material_types': ['http://localhost:8000/api/materialtype/47f94409-4086-4958-b3af-04575435320b/',
  'http://localhost:8000/api/materialtype/bdd47093-0170-4e3b-9290-4b2037382b6f/',
  'http://localhost:8000/api/materialtype/6cabb634-4c65-400c-8f3c-9f5a9afa88f1/'],
 'property': []}

These are the fields available for materials. Notably we can associate a material with arbitrary properties and material types.

### POST a material property

#### Current property definitions in ESCALATE

ESCALATE supports user defined property definitions, these are the one's we're using

In [12]:
r = get_data('propertydef', 
             {'fields': ['description']} # we can select 'columns' with the fields parameter    
            )
r

<Response [200]>
GET: OK
Found 16 resources, returning list of dicts)


[{'description': 'particle-size {min, max}'},
 {'description': 'mesh {min, max}'},
 {'description': 'capacity'},
 {'description': 'cross-linkage %'},
 {'description': 'moisture % {min, max}'},
 {'description': 'Resin Class'},
 {'description': 'Functional group'},
 {'description': 'concentration_molarity'},
 {'description': 'manufacturer'},
 {'description': 'plate well count'},
 {'description': 'plate well location'},
 {'description': 'plate well robot order'},
 {'description': 'plate well volume {min, max}'},
 {'description': 'concentration'},
 {'description': 'molecular-weight'},
 {'description': 'Crystal Score'}]

#### Molecular Weight

In [13]:
mw_property_def = get_data('propertydef', 
                           {'description': 'molecular-weight', # find an entity that has a particular description
                            'fields': ['url', 'description', 'val_unit']})
mw_property_def

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/propertydef/85eb75b1-a383-4dbb-bf07-e352407cdc6e/',
 'description': 'molecular-weight',
 'val_unit': 'g/mol'}

Now we can associate an instance of this property with a material, say EthNH3I

In [14]:
ethylammonium_iodide = get_data('material', 
                                {'description': 'Ethylammonium Iodide',
                                 'fields':['url', 'description', 'property']})
ethylammonium_iodide

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/9caa7a61-663a-4ea0-a3bf-712e80f566f5/',
 'description': 'Ethylammonium Iodide',
 'property': []}

Note the empty property list above

Lets fill that in with the molecular weight from PubChem

In [15]:
response = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/'+\
                        f'compound/cid/{PubChemIDs.get("Ethylammonium Iodide")}/'+\
                        'property/MolecularFormula,MolecularWeight,CanonicalSMILES/JSON')
                      # proxies=proxies)
pubchem_json = response.json()
eth_mw = pubchem_json['PropertyTable']['Properties'][0]['MolecularWeight']
eth_mw

'173.00'

In [16]:
r = post_data('materialproperty',
                  {'material': ethylammonium_iodide['url'],
                   'property_def': mw_property_def['url'],
                   'value': f"{eth_mw}"
                  }
                 )

<Response [201]>
POST: OK, returning new resource dict


And we've stored it!

In [17]:
get_data('material', 
         {'description':'Ethylammonium Iodide'})

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/9caa7a61-663a-4ea0-a3bf-712e80f566f5/',
 'uuid': '9caa7a61-663a-4ea0-a3bf-712e80f566f5',
 'edocs': [],
 'tags': [],
 'notes': [],
 'description': 'Ethylammonium Iodide',
 'consumable': True,
 'composite_flg': False,
 'material_class': 'model',
 'actor_description': None,
 'status_description': 'active',
 'add_date': '2021-05-18T18:36:39.541032Z',
 'mod_date': '2021-05-18T18:36:48.001730Z',
 'actor': None,
 'status': 'http://localhost:8000/api/status/5f7417a3-cf9a-425d-b4c5-f72db9c5d702/',
 'material_types': ['http://localhost:8000/api/materialtype/47f94409-4086-4958-b3af-04575435320b/',
  'http://localhost:8000/api/materialtype/bdd47093-0170-4e3b-9290-4b2037382b6f/'],
 'property': ['http://localhost:8000/api/property/ffcb62bf-9f8d-44ca-ab54-cc6da4b058c1/']}


* In practice we can use this functionality to store properties from any experiment or calculation. 

* We can also store metadata about where these values came from   
  (example to come in a further tutorial on tags, notes, edocs, calculations).

## Action definitions

* Just as we are free to define properties, we are free to define actions
* Current definition are what we've needed to specify human/robot instructions for current workflows

In [18]:
r = get_data('actiondef', {'fields': ['description', 'uuid', 'url']})
r

<Response [200]>
GET: OK
Found 7 resources, returning list of dicts)


[{'url': 'http://localhost:8000/api/actiondef/cbd005a2-c65e-4540-a7d3-c32ce179d1f3/',
  'uuid': 'cbd005a2-c65e-4540-a7d3-c32ce179d1f3',
  'description': 'dispense'},
 {'url': 'http://localhost:8000/api/actiondef/7d4ef633-6629-4a37-bf18-c6989d6b1735/',
  'uuid': '7d4ef633-6629-4a37-bf18-c6989d6b1735',
  'description': 'heat_stir'},
 {'url': 'http://localhost:8000/api/actiondef/469e7e6e-b362-4550-93fd-0774114a139b/',
  'uuid': '469e7e6e-b362-4550-93fd-0774114a139b',
  'description': 'heat'},
 {'url': 'http://localhost:8000/api/actiondef/5cf1b3af-2a90-45b7-9afd-efe310dba679/',
  'uuid': '5cf1b3af-2a90-45b7-9afd-efe310dba679',
  'description': 'start_node'},
 {'url': 'http://localhost:8000/api/actiondef/6779b28b-acfc-4f66-91b7-29c0f61adbca/',
  'uuid': '6779b28b-acfc-4f66-91b7-29c0f61adbca',
  'description': 'end_node'},
 {'url': 'http://localhost:8000/api/actiondef/41882d91-bca1-47e9-a939-1a5065d2f79d/',
  'uuid': '41882d91-bca1-47e9-a939-1a5065d2f79d',
  'description': 'dispense_solid'},

#### Zooming in on the dispense action definition

In [19]:
get_data('actiondef',               
         {'description': 'dispense',  # which action def
          'expand': 'parameter_def',  # sub dictionary to expand
          })

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/actiondef/cbd005a2-c65e-4540-a7d3-c32ce179d1f3/',
 'uuid': 'cbd005a2-c65e-4540-a7d3-c32ce179d1f3',
 'edocs': [],
 'tags': [],
 'notes': [{'url': 'http://localhost:8000/api/note/bf0ad585-f96c-459c-9648-0075a4f6f54b/',
   'uuid': 'bf0ad585-f96c-459c-9648-0075a4f6f54b',
   'notetext': 'source material = material to be dispensed, destination material = plate well',
   'add_date': '2021-05-18T18:36:44.813095Z',
   'mod_date': '2021-05-18T18:36:44.813095Z',
   'actor_description': 'Mike Tynes',
   'ref_note_uuid': 'cbd005a2-c65e-4540-a7d3-c32ce179d1f3',
   'actor': 'http://localhost:8000/api/actor/660a8d22-84a7-4672-8cfc-cbf9b01d9638/'}],
 'parameter_def': [{'url': 'http://localhost:8000/api/parameterdef/0b11e810-eecb-4a30-82be-7c6d9fc6cdef/',
   'uuid': '0b11e810-eecb-4a30-82be-7c6d9fc6cdef',
   'edocs': [],
   'tags': [],
   'notes': [],
   'description': 'volume',
   'default_val': {'value': 0.0, 'unit': 'mL', 'type': 'num'},
   'unit_type': 'volume',
  

### Actions + Materials = Experiment Template 

* Experiment template = the form of an experiment that I wish to re-use, varying material choices and process parameters

In [20]:
experiment_templates = get_data('experimenttemplate', 
                               {'fields':['description', 'url']})

<Response [200]>
GET: OK
Found 4 resources, returning list of dicts)


In [21]:
experiment_templates

[{'url': 'http://localhost:8000/api/experimenttemplate/0e0cf7e4-c97c-4615-8e70-29f7008644a5/',
  'description': 'liquid_solid_extraction'},
 {'url': 'http://localhost:8000/api/experimenttemplate/5ab64466-c4f4-4c23-9c89-846f9663e4b3/',
  'description': 'resin_weighing'},
 {'url': 'http://localhost:8000/api/experimenttemplate/5c2ac7f4-0047-40f9-9433-433e8f94d804/',
  'description': 'perovskite_demo'},
 {'url': 'http://localhost:8000/api/experimenttemplate/f89625e0-1836-4d35-adc3-607af320545e/',
  'description': 'test_wf_1'}]

Click on perovskite demo link and note 2 main nested fields: 
* Bill of materials = the initial materials for the perovskite workflow
* Workflow = the set of actions that combine these materials into perovskite crystal trials

In [22]:
perovskite_template = get_data('experimenttemplate',
                              {'description': 'test_wf_1', 
                              'expand': 'workflow' # expand the workflow subdictionary
                              })

<Response [200]>
GET: OK
Found one resource, returning dict


In [23]:
perovskite_template

{'url': 'http://localhost:8000/api/experimenttemplate/f89625e0-1836-4d35-adc3-607af320545e/',
 'uuid': 'f89625e0-1836-4d35-adc3-607af320545e',
 'edocs': [],
 'tags': [],
 'notes': [],
 'bill_of_materials': [{'url': 'http://localhost:8000/api/billofmaterials/c4e9ce7c-6620-40e9-95be-5d1922f1c050/',
   'uuid': 'c4e9ce7c-6620-40e9-95be-5d1922f1c050',
   'edocs': [],
   'tags': [],
   'notes': [],
   'bom_material': ['http://localhost:8000/api/bommaterial/30e78eff-9b9c-49fd-a962-d486076160f8/',
    'http://localhost:8000/api/bommaterial/cf28973e-f9c3-4043-ac3d-1a1a0ea04694/',
    'http://localhost:8000/api/bommaterial/3dedad03-5b47-4912-97b9-2052e11e5934/',
    'http://localhost:8000/api/bommaterial/63dbfb7d-435c-470e-9229-d294465a2636/',
    'http://localhost:8000/api/bommaterial/44f27922-ac0a-4650-a658-45c8e64f229c/'],
   'description': 'Test WF1 Materials',
   'experiment_description': 'test_wf_1',
   'add_date': '2021-05-18T18:36:48.143650Z',
   'mod_date': '2021-05-18T18:36:48.143650Z'

## Bill of Materials

The initial materials: think list of materials in methods section of paper

These are the Bill of Materials' Material entries for this experiment: 

In [24]:
get_data('bommaterial', {'bom':perovskite_template['bill_of_materials'][0]['uuid'], 
                        'fields':['description']})

<Response [200]>
GET: OK
Found 5 resources, returning list of dicts)


[{'description': 'Plate'},
 {'description': 'Stock A'},
 {'description': 'Stock B'},
 {'description': 'Acid'},
 {'description': 'Solvent'}]

In [25]:
get_data('compositematerial', {'composite_description__startswith':'Stock', 
                               'fields': ['composite_description',
                                          'component_description']})

<Response [200]>
GET: OK
Found 7 resources, returning list of dicts)


[{'composite_description': 'Stock A',
  'component_description': 'Lead Diiodide'},
 {'composite_description': 'Stock A',
  'component_description': 'Ethylammonium Iodide'},
 {'composite_description': 'Stock A',
  'component_description': 'Gamma-Butyrolactone'},
 {'composite_description': 'Stock B',
  'component_description': 'Ethylammonium Iodide'},
 {'composite_description': 'Stock B',
  'component_description': 'Gamma-Butyrolactone'},
 {'composite_description': 'Stock FAH',
  'component_description': 'Formic Acid'},
 {'composite_description': 'Stock FAH', 'component_description': 'Water'}]

#### We could drill further to get the concentrations, properties, etc

### Workflows: Logical groups of actions

Each of these contains a set of parameters that we can edit

In [26]:
perovskite_demo = get_data('experimenttemplate',
         {'description': 'perovskite_demo',
          'expand': 'workflow'})
[wf['description'] for wf in perovskite_demo['workflow']]

<Response [200]>
GET: OK
Found one resource, returning dict


['Perovskite Demo: Preheat Plate',
 'Perovskite Demo: Prepare Stock A',
 'Perovskite Demo: Prepare Stock B',
 'Perovskite Demo: Dispense Solvent',
 'Perovskite Demo: Dispense Stock A',
 'Perovskite Demo: Dispense Stock B',
 'Perovskite Demo: Dispense Acid Vol 1',
 'Perovskite Demo: Heat Stir 1',
 'Perovskite Demo: Dispense Acid Vol 2',
 'Perovskite Demo: Heat Stir 2',
 'Perovskite Demo: Heat']

In [27]:
dispense_solvent_wf = perovskite_demo['workflow'][3] # pull out a workflow
example_steps = dispense_solvent_wf['step'][4]     # pull out some steps
example_steps

{'url': 'http://localhost:8000/api/workflowstep/9324dbb0-e061-4cf7-ac8b-d8f6c3d3c70a/',
 'uuid': '9324dbb0-e061-4cf7-ac8b-d8f6c3d3c70a',
 'edocs': [],
 'tags': [],
 'notes': [],
 'workflow_object': 'http://localhost:8000/api/workflowobject/a97837c5-02cb-4552-8127-c5224f57f481/',
 'workflow_description': 'Perovskite Demo: Dispense Solvent',
 'parent_object_type': 'action',
 'parent_object_description': 'Perovskite Demo: Dispense Solvent: Solvent -> Plate: 96 Well Plate well#: A4',
 'parent_path': None,
 'conditional_val': None,
 'conditional_value': None,
 'status_description': None,
 'add_date': '2021-05-18T18:36:50.323131Z',
 'mod_date': '2021-05-18T18:36:50.323131Z',
 'workflow': 'http://localhost:8000/api/workflow/ad1d32a9-e49e-4e45-b927-3446ca1329ef/',
 'parent': 'http://localhost:8000/api/workflowstep/75006f77-bfcb-45b8-9b27-81b41dea9c78/',
 'status': None}

## Creating a new workflow from a template

If I want create an instance experiment from a template I: 

In [28]:
editable_template = get_data(
    # template endpoint  /        template ID      / create an instance of this template 
    f'experimenttemplate/{perovskite_demo["uuid"]}/create'
)

<Response [200]>
GET: OK


In [29]:
editable_template

{'experiment_name': '',
 'material_parameters': [{'material_name': 'Neat GBL',
   'value': 'http://localhost:8000/api/inventorymaterial/afff158e-6875-4dfb-a265-d89579d4752b/'},
  {'material_name': 'Lead Diiodide',
   'value': 'http://localhost:8000/api/inventorymaterial/6a2d3dc2-ada4-4864-b54c-74863782ce08/'},
  {'material_name': 'Ethylammonium Iodide',
   'value': 'http://localhost:8000/api/inventorymaterial/d0951677-ab50-4728-bd53-532e01c27c05/'},
  {'material_name': 'Plate: 96 Well',
   'value': 'http://localhost:8000/api/inventorymaterial/98afab9d-2091-45f3-8067-b3d2f2a8e2af/'},
  {'material_name': 'Stock FAH',
   'value': 'http://localhost:8000/api/inventorymaterial/e5e47800-96df-4f18-af04-99814faab4ac/'},
  {'material_name': 'Tube: 5mL',
   'value': 'http://localhost:8000/api/inventorymaterial/d47fae6f-485d-4d4d-a985-d3c81cba78ea/'},
  {'material_name': 'Tube: 5mL',
   'value': 'http://localhost:8000/api/inventorymaterial/d47fae6f-485d-4d4d-a985-d3c81cba78ea/'}],
 'experiment_par

In [30]:
editable_template['experiment_name'] = 'test_perovskite_instance_updated'

Suppose I edit this json then I can post the new template to the server

In [31]:
%%time
resp = post_data(
    f'experimenttemplate/{perovskite_demo["uuid"]}/create',
    editable_template
)

<Response [200]>
POST: OK, returning new resource dict
CPU times: user 3.29 ms, sys: 2.3 ms, total: 5.59 ms
Wall time: 4.56 s


In [32]:
resp

{'new_experiment_created': 'http://localhost:8000/api/experiment/59c19e0e-fafd-4562-a6af-73b24d7778d4/'}

This experiment then appears in the experiment queue.   

http://localhost:8000/experiment_list

The experimentalist is notified, can download relevant robot input and upload observed values through forms

In [33]:
%%time
new_experiment_json = get_data('experiment/' + resp['new_experiment_created'].split('/')[-2], 
                                {'expand': 'workflow.step.workflow_object.action.parameter'}) # expanding deeply nested fields is somewhat slow

<Response [200]>
GET: OK
CPU times: user 29 ms, sys: 7.9 ms, total: 36.9 ms
Wall time: 35.8 s


In [34]:
experiment_json = [new_experiment_json]

In [35]:
experiment_json

[{'url': 'http://localhost:8000/api/experiment/59c19e0e-fafd-4562-a6af-73b24d7778d4/',
  'uuid': '59c19e0e-fafd-4562-a6af-73b24d7778d4',
  'edocs': [],
  'tags': [],
  'notes': [],
  'bill_of_materials': [{'url': 'http://localhost:8000/api/billofmaterials/7091097a-f10b-4f4e-8d31-3aae89d617e7/',
    'uuid': '7091097a-f10b-4f4e-8d31-3aae89d617e7',
    'edocs': [],
    'tags': [],
    'notes': [],
    'bom_material': ['http://localhost:8000/api/bommaterial/5e0ed658-09d4-456e-aa2f-ba4c6d250707/',
     'http://localhost:8000/api/bommaterial/8131d14d-4f1a-465f-a264-1c608a78198e/',
     'http://localhost:8000/api/bommaterial/a1f664b6-190e-4bb2-9692-8fbdf5bc3045/',
     'http://localhost:8000/api/bommaterial/6951ffb8-9da5-4363-a3a1-563655a4b18b/',
     'http://localhost:8000/api/bommaterial/2e82ab4f-9a5f-42c9-bccd-24b7a9187cb1/',
     'http://localhost:8000/api/bommaterial/f7f9af7d-fe9f-4961-bf28-5448d495eec3/',
     'http://localhost:8000/api/bommaterial/1f72271c-5907-40ac-890e-f245aed18189/'

In [36]:
def experiment_json_to_df(experiment_json):
    result = []
    for e in experiment_json: 
        for workflow in e['workflow']:
            if 'Dispense' not in workflow['description']:
                    continue
            for step in workflow['step']: 
                action = step['workflow_object']['action']
                for parameter in action['parameter']:
                    result.append(
                        dict(
                            experiment_url         = e['url'],
                            experiment_id          = e['url'].split('/')[-2],
                            action_source          = action['source_material_description'],
                            action_dest            = action['destination_material_description'],
                            action_parameter       = parameter['parameter_def_description'],
                            action_parameter_value = float(parameter['parameter_val_nominal']['value']),
                            action_parameter_unit  = parameter['parameter_val_nominal']['unit']
                            )
                        )
    return pd.DataFrame(result)

In [37]:
results = experiment_json_to_df(experiment_json)

In [38]:
results

,experiment_url,experiment_id,action_source,action_dest,action_parameter,action_parameter_value,action_parameter_unit
0,http://localhost:8000/api/experiment/59c19e0e-...,59c19e0e-fafd-4562-a6af-73b24d7778d4,Solvent,Plate: 96 Well Plate well#: E3,volume,0.0,mL
1,http://localhost:8000/api/experiment/59c19e0e-...,59c19e0e-fafd-4562-a6af-73b24d7778d4,Solvent,Plate: 96 Well Plate well#: H11,volume,0.0,mL
2,http://localhost:8000/api/experiment/59c19e0e-...,59c19e0e-fafd-4562-a6af-73b24d7778d4,Solvent,Plate: 96 Well Plate well#: D11,volume,0.0,mL
3,http://localhost:8000/api/experiment/59c19e0e-...,59c19e0e-fafd-4562-a6af-73b24d7778d4,Solvent,Plate: 96 Well Plate well#: C10,volume,0.0,mL
4,http://localhost:8000/api/experiment/59c19e0e-...,59c19e0e-fafd-4562-a6af-73b24d7778d4,Solvent,Plate: 96 Well Plate well#: E6,volume,0.0,mL
...,...,...,...,...,...,...,...
475,http://localhost:8000/api/experiment/59c19e0e-...,59c19e0e-fafd-4562-a6af-73b24d7778d4,Acid,Plate: 96 Well Plate well#: H4,volume,0.0,mL
476,http://localhost:8000/api/experiment/59c19e0e-...,59c19e0e-fafd-4562-a6af-73b24d7778d4,Acid,Plate: 96 Well Plate well#: G1,volume,0.0,mL
477,http://localhost:8000/api/experiment/59c19e0e-...,59c19e0e-fafd-4562-a6af-73b24d7778d4,Acid,Plate: 96 Well Plate well#: F9,volume,0.0,mL
478,http://localhost:8000/api/experiment/59c19e0e-...,59c19e0e-fafd-4562-a6af-73b24d7778d4,Acid,Plate: 96 Well Plate well#: A3,volume,0.0,mL


In [39]:
results = results.pivot_table(index=['experiment_id', 'action_dest'], 
                             columns=['action_source'], 
                             values='action_parameter_value')
results

action_source                                                         Acid  \
experiment_id                        action_dest                             
59c19e0e-fafd-4562-a6af-73b24d7778d4 Plate: 96 Well Plate well#: A1    0.0   
                                     Plate: 96 Well Plate well#: A10   0.0   
                                     Plate: 96 Well Plate well#: A11   0.0   
                                     Plate: 96 Well Plate well#: A12   0.0   
                                     Plate: 96 Well Plate well#: A2    0.0   
...                                                                    ...   
                                     Plate: 96 Well Plate well#: H5    0.0   
                                     Plate: 96 Well Plate well#: H6    0.0   
                                     Plate: 96 Well Plate well#: H7    0.0   
                                     Plate: 96 Well Plate well#: H8    0.0   
                                     Plate: 96 Well Plate well#: H9    0.0   

action_source                                                         Solvent  \
experiment_id                        action_dest                                
59c19e0e-fafd-4562-a6af-73b24d7778d4 Plate: 96 Well Plate well#: A1       0.0   
                                     Plate: 96 Well Plate well#: A10      0.0   
                                     Plate: 96 Well Plate well#: A11      0.0   
                                     Plate: 96 Well Plate well#: A12      0.0   
                                     Plate: 96 Well Plate well#: A2       0.0   
...                                                                       ...   
                                     Plate: 96 Well Plate well#: H5       0.0   
                                     Plate: 96 Well Plate well#: H6       0.0   
                                     Plate: 96 Well Plate well#: H7       0.0   
                                     Plate: 96 Well Plate well#: H8       0.0   
                                     Plate: 96 Well Plate well#: H9       0.0   

action_source                                                         Stock A  \
experiment_id                        action_dest                                
59c19e0e-fafd-4562-a6af-73b24d7778d4 Plate: 96 Well Plate well#: A1       0.0   
                                     Plate: 96 Well Plate well#: A10      0.0   
                                     Plate: 96 Well Plate well#: A11      0.0   
                                     Plate: 96 Well Plate well#: A12      0.0   
                                     Plate: 96 Well Plate well#: A2       0.0   
...                                                                       ...   
                                     Plate: 96 Well Plate well#: H5       0.0   
                                     Plate: 96 Well Plate well#: H6       0.0   
                                     Plate: 96 Well Plate well#: H7       0.0   
                                     Plate: 96 Well Plate well#: H8       0.0   
                                     Plate: 96 Well Plate well#: H9       0.0   

action_source                                                         Stock B  
experiment_id                        action_dest                               
59c19e0e-fafd-4562-a6af-73b24d7778d4 Plate: 96 Well Plate well#: A1       0.0  
                                     Plate: 96 Well Plate well#: A10      0.0  
                                     Plate: 96 Well Plate well#: A11      0.0  
                                     Plate: 96 Well Plate well#: A12      0.0  
                                     Plate: 96 Well Plate well#: A2       0.0  
...                                                                       ...  
                                     Plate: 96 Well Plate well#: H5       0.0  
                                     Plate: 96 Well Plate well#: H6       0.0  
                                     Plate: 96 Well Plate well#: H7       0.0

In [40]:
crystal_scores = get_data('measure', 
                          {'measuredef':
                                (get_data('measuredef', 
                                          {'description': 'crystal_score'})['url']
                                )
                          })                        
results['crystal_score'] = crystal_scores['measure_value']['value']

<Response [200]>
GET: OK
Found one resource, returning dict
<Response [200]>
GET: OK
Found one resource, returning dict


In [41]:
results

action_source                                                         Acid  \
experiment_id                        action_dest                             
59c19e0e-fafd-4562-a6af-73b24d7778d4 Plate: 96 Well Plate well#: A1    0.0   
                                     Plate: 96 Well Plate well#: A10   0.0   
                                     Plate: 96 Well Plate well#: A11   0.0   
                                     Plate: 96 Well Plate well#: A12   0.0   
                                     Plate: 96 Well Plate well#: A2    0.0   
...                                                                    ...   
                                     Plate: 96 Well Plate well#: H5    0.0   
                                     Plate: 96 Well Plate well#: H6    0.0   
                                     Plate: 96 Well Plate well#: H7    0.0   
                                     Plate: 96 Well Plate well#: H8    0.0   
                                     Plate: 96 Well Plate well#: H9    0.0   

action_source                                                         Solvent  \
experiment_id                        action_dest                                
59c19e0e-fafd-4562-a6af-73b24d7778d4 Plate: 96 Well Plate well#: A1       0.0   
                                     Plate: 96 Well Plate well#: A10      0.0   
                                     Plate: 96 Well Plate well#: A11      0.0   
                                     Plate: 96 Well Plate well#: A12      0.0   
                                     Plate: 96 Well Plate well#: A2       0.0   
...                                                                       ...   
                                     Plate: 96 Well Plate well#: H5       0.0   
                                     Plate: 96 Well Plate well#: H6       0.0   
                                     Plate: 96 Well Plate well#: H7       0.0   
                                     Plate: 96 Well Plate well#: H8       0.0   
                                     Plate: 96 Well Plate well#: H9       0.0   

action_source                                                         Stock A  \
experiment_id                        action_dest                                
59c19e0e-fafd-4562-a6af-73b24d7778d4 Plate: 96 Well Plate well#: A1       0.0   
                                     Plate: 96 Well Plate well#: A10      0.0   
                                     Plate: 96 Well Plate well#: A11      0.0   
                                     Plate: 96 Well Plate well#: A12      0.0   
                                     Plate: 96 Well Plate well#: A2       0.0   
...                                                                       ...   
                                     Plate: 96 Well Plate well#: H5       0.0   
                                     Plate: 96 Well Plate well#: H6       0.0   
                                     Plate: 96 Well Plate well#: H7       0.0   
                                     Plate: 96 Well Plate well#: H8       0.0   
                                     Plate: 96 Well Plate well#: H9       0.0   

action_source                                                         Stock B  \
experiment_id                        action_dest                                
59c19e0e-fafd-4562-a6af-73b24d7778d4 Plate: 96 Well Plate well#: A1       0.0   
                                     Plate: 96 Well Plate well#: A10      0.0   
                                     Plate: 96 Well Plate well#: A11      0.0   
                                     Plate: 96 Well Plate well#: A12      0.0   
                                     Plate: 96 Well Plate well#: A2       0.0   
...                                                                       ...   
                                     Plate: 96 Well Plate well#: H5       0.0   
                                     Plate: 96 Well Plate well#: H6       0.0   
                                     Plate: 96 Well Plate well#: H7

### Current Limitations

* Some parts of API still are 'high entropy' (e.g. measure)
* Ditto for some portions of UI
* REST is slow for large transfers